In [1]:
import numpy as np
import pickle
from astropy.io import fits
import matplotlib.pyplot as plt
from galpy.util import bovy_conversion, bovy_coords, save_pickles, bovy_plot
from galpy.potential import MWPotential2014, turn_physical_off, vcirc
import astropy.units as u
%pylab inline
from galpy.orbit import Orbit



ro=8.0
#paper on MC used R0=8.5 kpc, using ro=8. as of now.
vo=220.

/home/nil/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters

Populating the interactive namespace from numpy and matplotlib


In [2]:
hdulist=fits.open('molecular_clouds/J_ApJ_834_57_table1.dat.gz.fits')
print (hdulist[1].columns)
aa=hdulist[1].data

ColDefs(
    name = 'Cloud'; format = 'I4'; disp = 'I4'; start = 1
    name = 'Ncomp'; format = 'I3'; disp = 'I3'; start = 6
    name = 'Npix'; format = 'I3'; disp = 'I3'; start = 10
    name = 'Area'; format = 'E12.6'; unit = 'deg2'; disp = 'E12.6'; start = 14
    name = 'GLON'; format = 'E13.6'; unit = 'deg'; disp = 'E13.6'; start = 27
    name = 'Sigb'; format = 'E12.6'; unit = 'deg'; disp = 'E12.6'; start = 41
    name = 'GLAT'; format = 'E13.6'; unit = 'deg'; disp = 'E13.6'; start = 54
    name = 'Sigl'; format = 'E12.6'; unit = 'deg'; disp = 'E12.6'; start = 68
    name = 'theta'; format = 'E13.6'; unit = 'deg'; disp = 'E13.6'; start = 81
    name = 'WCO'; format = 'E12.6'; unit = 'K.km/s'; disp = 'E12.6'; start = 95
    name = 'NH2'; format = 'E12.6'; unit = 'cm-2'; disp = 'E12.6'; start = 108
    name = 'Sigma'; format = 'E12.6'; unit = 'Msun/pc2'; disp = 'E12.6'; start = 121
    name = 'Vcent'; format = 'E13.6'; unit = 'km/s'; disp = 'E13.6'; start = 134
    name = 'SigV'; for

#To setup the orbits of the MCs, take (l,b,d) --> galactocentric rectangular coordinates

[R,vR,vT(,z,vz,phi)]

In [3]:
l=aa['GLON']
b=aa['GLAT']
#Near or far distance flag (0=near; 1=far) 
flag=aa['INF']
Dnear=aa['Dnear']
Dfar=aa['Dfar']
znear=aa['znear']
zfar=aa['zfar']
R_sph_gal=aa['Rgal']

In [4]:
def lbd_to_galcencyl(l,b,d,degree=True):
    xyz=bovy_coords.lbd_to_XYZ(l,b,d,degree=degree)
    Rphiz=bovy_coords.XYZ_to_galcencyl(xyz[:,0],xyz[:,1],xyz[:,2],Xsun=1.,Zsun=0.)
    
    return (Rphiz[:,0],Rphiz[:,1],Rphiz[:,2])

D=np.empty(len(l))
zfile=np.empty(len(l))

for ii in range(len(l)):
    if flag[ii] == 0 :
        D[ii]=Dnear[ii]
        zfile[ii]=znear[ii]
        
    else :
        D[ii]=Dfar[ii]
        zfile[ii]=zfar[ii]
        


R,phi,z= lbd_to_galcencyl(l,b,D)

R/=ro
z/=ro

print (min(phi),max(phi))

0.00034008007131385915 6.282280181831788


In [5]:
print (max(np.abs(z*ro-zfile)))  #computed z and z from catalog match pretty well

0.003307787684446284


In [6]:
vT=np.empty(len(l))

for ii in range(len(l)):
    vT[ii]=vcirc(MWPotential2014,R[ii])

print (vT)

vR=np.zeros(len(l))
vz=np.zeros(len(l))


[1.00553195 1.00588521 1.00468541 ... 0.92576543 0.90723473 1.0217835 ]


In [7]:
#initialize orbits of the MCs and integrate them back 5 Gyrs and then integrete them forward for 5 Gyr


t_age= np.linspace(0.,5.,1001)*u.Gyr/bovy_conversion.time_in_Gyr(vo,ro)
orbits= []

N=10 #len(l)

for ii in range(N):
    o= Orbit([R[ii],vR[ii],vT[ii],z[ii],vz[ii],phi[ii]]).flip() # flip flips the velocities for backwards integration
    
    o.integrate(t_age,MWPotential2014)
    orbits.append(o(t_age[-1]).flip()) 
    
    orbits[ii].integrate(t_age,MWPotential2014)

In [ ]:

orbits[0].plot(d1='x',d2='y',ro=ro,vo=vo)

In [ ]:
orbits[1].plot('k.',d1='ra',d2='dec',ro=ro,vo=vo)

In [8]:
#load the saved pickle file for converting apars to obs coords
with open('/home/nil/Dropbox/streamgap-pepper/pal5_64sampling.pkl','rb') as savefile:
            sdf_smooth= pickle.load(savefile,encoding='latin1')
            sdf_pepper= pickle.load(savefile,encoding='latin1')

In [10]:
print (sdf_pepper._timpact)

[  2.16342958   4.32685916   6.49028874   8.65371832  10.8171479
  12.98057748  15.14400706  17.30743664  19.47086622  21.6342958
  23.79772538  25.96115496  28.12458454  30.28801412  32.4514437
  34.61487328  36.77830286  38.94173244  41.10516202  43.2685916
  45.43202118  47.59545076  49.75888034  51.92230992  54.0857395
  56.24916908  58.41259866  60.57602824  62.73945782  64.9028874
  67.06631698  69.22974656  71.39317614  73.55660572  75.7200353
  77.88346488  80.04689446  82.21032404  84.37375362  86.5371832
  88.70061278  90.86404236  93.02747194  95.19090152  97.3543311
  99.51776068 101.68119026 103.84461984 106.00804942 108.171479
 110.33490858 112.49833816 114.66176774 116.82519732 118.9886269
 121.15205648 123.31548606 125.47891564 127.64234522 129.8057748
 131.96920438 134.13263396 136.29606354 138.45949311]


In [15]:
import simulate_streampepper

timpact= simulate_streampepper.parse_times('64sampling',5.)
print (timpact)
#timpact = list(np.linspace(0.1,0.5,10)/bovy_conversion.time_in_Gyr(vo,ro))



print (np.array(timpact)*bovy_conversion.time_in_Gyr(vo,ro))

x=[]
y=[]
z=[]

for kk in range(len(timpact)):
    a= sdf_pepper._sgapdfs_coordtransform[timpact[kk]]._kick_interpolatedObsTrackXY
    
    x.append(a[:,0])
    y.append(a[:,1])
    z.append(a[:,2])
    apar_all=sdf_pepper._sgapdfs_coordtransform[timpact[kk]]._kick_interpolatedThetasTrack
    #print (max(apar_all))
    apar_max=sdf_smooth.length(tdisrupt=sdf_pepper._tdisrupt-timpact[kk])*sdf_pepper._length_factor
    print (apar_max)
    #apar=apar_all[apar_all < apar_max]
    #print len(apar)
    


[2.163429579921305, 4.32685915984261, 6.490288739763915, 8.65371831968522, 10.817147899606525, 12.98057747952783, 15.144007059449136, 17.30743663937044, 19.470866219291743, 21.63429579921305, 23.797725379134356, 25.96115495905566, 28.12458453897696, 30.28801411889827, 32.451443698819574, 34.61487327874088, 36.77830285866218, 38.941732438583486, 41.10516201850479, 43.2685915984261, 45.4320211783474, 47.59545075826871, 49.75888033819001, 51.92230991811132, 54.08573949803262, 56.24916907795392, 58.412598657875236, 60.57602823779654, 62.73945781771784, 64.90288739763915, 67.06631697756045, 69.22974655748176, 71.39317613740305, 73.55660571732436, 75.72003529724566, 77.88346487716697, 80.04689445708827, 82.21032403700958, 84.37375361693088, 86.5371831968522, 88.7006127767735, 90.8640423566948, 93.0274719366161, 95.19090151653742, 97.35433109645872, 99.51776067638002, 101.68119025630132, 103.84461983622263, 106.00804941614393, 108.17147899606525, 110.33490857598653, 112.49833815590785, 114.66

In [ ]:
apar_max=sdf_pepper.length(tdisrupt=sdf_pepper._tdisrupt-timpact[0])*sdf_pepper._length_factor

print (apar_max)

In [ ]:
print len(x[0])    
lbd=bovy_coords.XYZ_to_lbd(x[0],y[0],z[0],degree=True)

print len(lbd[:,0])

In [ ]:
print sdf_pepper.length(tdisrupt=sdf_pepper._tdisrupt-timpact[0])#*sdf_pepper._length_factor

In [ ]:
t_age= np.linspace(0.,5.,1001)*u.Gyr

pal5= Orbit([229.018,-0.124,23.2,-2.296,-2.257,-58.7],radec=True,ro=ro,vo=vo,solarmotion=[-11.1,24.,7.25])

#pal5.integrate(t_age,MWPotential2014)
#pal5(t_age[-1]).flip()

t=np.linspace(0.,0.4,1001)*u.Gyr#/bovy_conversion.time_in_Gyr(vo,ro)

pal5.integrate(t,MWPotential2014)

In [ ]:
bovy_plot.bovy_print(axes_labelsize=18.,xtick_labelsize=14.,ytick_labelsize=14.)

#pal5.plot(d1='x',d2='y')
#pal5.plot()

pal5.plot(d1='x',d2='y')

In [ ]:
import simulate_streampepper

def galcencyl_to_lbd(R,phi,Z,degree=True):
    xyz=bovy_coords.galcencyl_to_XYZ(R,phi,Z)
    lbd=bovy_coords.XYZ_to_lbd(xyz[0],xyz[1],xyz[2],degree=degree)
    return lbd[0], lbd[1], lbd[2]

timp= np.array(timpact)*bovy_conversion.time_in_Gyr(vo,ro)

print timp

pal5x= pal5.x(timp*u.Gyr)
pal5y=pal5.y(timp*u.Gyr)
pal5z=pal5.z(timp*u.Gyr)

print pal5x
#pal5_z=pal5.z(np.array(timpact)*u.Gyr)

In [ ]:
bovy_plot.bovy_print(axes_labelsize=18.,xtick_labelsize=14.,ytick_labelsize=14.)


scatter(pal5x[0],pal5y[0])
scatter(x[0]*ro,y[0]*ro)
pal5.plot(d1='x',d2='y',overplot=True,c='k')


In [ ]:
bovy_plot.bovy_print(axes_labelsize=18.,xtick_labelsize=14.,ytick_labelsize=14.)


#scatter(pal5x[0],pal5y[0])
scatter(pal5x[0],pal5z[0])
scatter(x[0]*ro,z[0]*ro)
pal5.plot(d1='x',d2='z',overplot=True,c='k')

In [ ]:
print lbd[0]

In [ ]:
plt.figure()
plt.plot(pal5_l,pal5_b)
plt.show()